## Imports

In [1]:
import torch
import open_clip
from torch import nn

import os
import argparse
import random
import numpy as np
import torch
from torch.nn import functional as F
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from CLIP.learnable_prompt import PromptMaker
from dataset.medical_zero import MedTestDataset, MedTrainDataset
from CLIP.clip import create_model
from CLIP.adapter import CLIP_Inplanted
from loss import FocalLoss, BinaryDiceLoss
from pathlib import Path
import warnings
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import random

/home/vvthai10/miniconda3/envs/mvfa-ad/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
biomedclip_hf_api = "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"

In [3]:
biomedclip = open_clip.create_model(biomedclip_hf_api)
biomedclip.eval()

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768

In [26]:
visual = biomedclip.visual.trunk
visual.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [27]:
resize = 224
transform_image = transforms.Compose(
    [
        transforms.Resize((resize, resize), Image.BICUBIC),
        transforms.ToTensor(),
    ]
)

In [30]:
img = Image.open("./data/Brain_AD/test/good/img/00000_96.png").convert("RGB")
img = transform_image(img)
img  = img.unsqueeze(0)
img = img.to("cuda")
img.shape

torch.Size([1, 3, 224, 224])

In [31]:
x = visual.patch_embed(img)
print(x.shape)
x = visual._pos_embed(x)
print(x.shape)
x = visual.norm_pre(x)
print(x.shape)

torch.Size([1, 196, 768])
torch.Size([1, 197, 768])
torch.Size([1, 197, 768])


In [32]:
x = visual.blocks[0](x)

In [33]:
x.shape

torch.Size([1, 197, 768])

In [8]:
biomedclip.text

HFTextEncoder(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [9]:
from open_clip import create_model_from_pretrained, get_tokenizer
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

In [13]:
context_length = 256
template = 'this is a photo of '
labels = [
    'adenocarcinoma histopathology',
    'brain MRI',
    'covid line chart',
    'squamous cell carcinoma histopathology',
    'immunohistochemistry histopathology',
    'bone X-ray',
    'chest X-ray',
    'pie chart',
    'hematoxylin and eosin histopathology'
]
texts = tokenizer([template + l for l in labels], context_length=context_length).to('cuda')

In [36]:
tokenizer

In [22]:
texts.shape

torch.Size([9, 256])

In [39]:
biomedclip.text.transformer.embeddings.LayerNorm

LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [37]:
biomedclip = biomedclip.to('cuda')
out = biomedclip.text.transformer.embeddings.position_embeddings(texts)

In [38]:
out.shape

torch.Size([9, 256, 768])

In [34]:
out = biomedclip.text.transformer(
            input_ids=texts,
        )

In [35]:
out.last_hidden_state.shape

torch.Size([9, 256, 768])